```markdown
Se utilizó los siguiente en neo4j para armar el RDD:

1. Instanciar el grafo: Merge, se generan los nodos y luego las relaciones entre esos nodos siguiendo la estrucutra (n1, label, n2)


// Crear nodos
MERGE (n1:Node {id: 1})
MERGE (n2:Node {id: 2})
MERGE (n3:Node {id: 3})
MERGE (n4:Node {id: 4})
MERGE (n5:Node {id: 5})
MERGE (n6:Node {id: 6})

// Crear relaciones
MERGE (n1)-[:REL {label: 11}]->(n2)
MERGE (n1)-[:REL {label: 11}]->(n3)
MERGE (n2)-[:REL {label: 11}]->(n3)
MERGE (n3)-[:REL {label: 11}]->(n2)
MERGE (n3)-[:REL {label: 11}]->(n4)
MERGE (n4)-[:REL {label: 11}]->(n1)
MERGE (n4)-[:REL {label: 11}]->(n2)
MERGE (n4)-[:REL {label: 11}]->(n3)
MERGE (n4)-[:REL {label: 12}]->(n5)
MERGE (n5)-[:REL {label: 12}]->(n1)
MERGE (n5)-[:REL {label: 12}]->(n2)
MERGE (n5)-[:REL {label: 12}]->(n6)


2. Ploteo del grafo: Se utiliza Match para definir las relaciones que se van a plotear en el grafo. En este caso (n)-[r]->(m) nos indica que debemos plotear todo el grafo.


MATCH (n)-[r]->(m)
RETURN n, r, m


3. Ploteo de triángulos: Siguiendo la misma lógica del Match anterior, ahora se filtra el ploteo del Match de modo que solo se muestren los nodos que cumplen (x,label,y), (y,label,z), (z,label,x)

MATCH (x)-[r1]->(y),
      (y)-[r2]->(z),
      (z)-[r3]->(x)
RETURN x, y, z


## Creando una RDD en Pyspark con un servidor levantado en Neo4j

In [ ]:
from pyspark.sql import SparkSession
from neo4j import GraphDatabase

# Configurar nombre de usuario y contraseña de Neo4j
neo4j_user = "neo4j"
neo4j_password = "your_password"

# Inicializa la sesión de Spark
spark = SparkSession.builder \
    .appName("Neo4j to Spark") \
    .config("spark.neo4j.bolt.url", "bolt://localhost:7687") \
    .config("spark.neo4j.bolt.user", neo4j_user) \
    .config("spark.neo4j.bolt.password", neo4j_password) \
    .getOrCreate()

# Función para conectar a Neo4j y obtener las aristas
def get_edges_from_neo4j():
    driver = GraphDatabase.driver("bolt://localhost:7687", auth=(neo4j_user, neo4j_password)) #Entregamos el puerto, y el nombre de usuario y contraseña.
    with driver.session() as session:
        result = session.run("MATCH (a)-[r]->(b) RETURN a.id AS node1, r.label AS label, b.id AS node2") #Notar que usamos el mismo tipo de consultas para Neo4j.
        edges = [(record["node1"], record["label"], record["node2"]) for record in result] #Aquí seguimos la estructura (n1, label, n2)
    driver.close()
    return edges

# Extrae las aristas del grafo de Neo4j
edges = get_edges_from_neo4j()

# Crea un RDD de Spark con las aristas (Usamos la función para el parallelize)
edges_rdd = spark.sparkContext.parallelize(edges)

# Muestra el contenido del RDD
for edge in edges_rdd.collect():
    print(edge)
